In [1]:
from google.colab import drive
import numpy as np
import os
from os import listdir
from os.path import isfile, join, splitext
from PIL import Image, ImageOps
import numpy as np
import random
import math
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.regularizers import l1
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import Input
from matplotlib import pyplot

In [4]:
#If you didn't run the previous code that do the preprocessing -> run this part of code please
#DATASET USED FOR THE MODEL FOR TEMPO ESTIMATION
!gdown --id 17T-UI5zb5WCBga4jZzKC-sF4z4loDFr1
!gdown --id 1MgrEAf7k2CM0POlwmrBuK4nJlp_ciIoY

path=['Image_spec2.zip','Annotations.zip']
path_images=["Image_spec2/"]
path_ann=["Annotations/"]
from zipfile import ZipFile
for p in path:
  with ZipFile(p, 'r') as zipp:
      # printing all the contents of the zip file
      #zip.printdir()
      print('Extracting all the files now...')
      zipp.extractall()
      print('Done!')
flag=0

Extracting all the files now...
Done!
Extracting all the files now...
Done!


In [ ]:
#If you have done the preprocessing using the previous notebook, please run this part of code
path_images=["Dataset/Key/GS/Image_spec","Dataset/Tempo/GTZAN/Image_spec"]
path_ann=["Dataset/Tempo/GS/Ann","Dataset/Tempo/GTZAN/Ann"]
flag=1

In [17]:
def split_train_test_validation(path_label,path_img):
  if flag==1:
    for i,l in zip(path_img,path_label):
      f_l.extend([os.path.join(root, name) for root, dirs, files in os.walk(l) for name in files if name.endswith((".txt"))])
      f_i.extend([os.path.join(root, name) for root, dirs, files in os.walk(i) for name in files if name.endswith((".jpeg"))])   
      f_l=[os.path.join(f.split('/')[1],f.split('/')[2]) for f in f_l]
      f_i=[os.path.join(f.split('/')[1],f.split('/')[2]) for f in f_i]
  else:
    f_l=[f for f in listdir(path_label[0]) if isfile(join(path_label[0], f))]
    f_i=[f for f in listdir(path_img[0]) if isfile(join(path_img[0], f))]
  f_l= np.asarray(f_l)
  f_i= np.asarray(f_i)
  print("Sorting both images and annotations...")
  for n in range(f_l.shape[0]):
    while True:
      s=splitext(f_l[n])[0]
      if "lerch" in s:
        s=s[:len(s)-6]
      res=np.where(f_i==f"{str(s)}.jpeg")
      if(n==res[0]):
        break
      f_i[n], f_i[res[0][0]] = f_i[res[0][0]], f_i[n]
  c=list(zip(f_l,f_i))
  random.shuffle(c)
  f_l,f_i=zip(*c)
  images=[]
  ann=[]
  print("Processing the images...")
  for f in f_i:
    images.append(np.asarray(ImageOps.grayscale(Image.open(f"Image_spec2/{f}").resize((100,100)))))
  print("Processing the annotations")
  for an in f_l:
    with open(f"Annotations/{an}") as f:
      ann.append(int(f.readlines()[0]))
  test_x=np.asarray(images[0:math.ceil(len(f_i)*0.2)])
  test_file=f_i[:math.ceil(len(f_i)*0.2)]
  test_y=np.array(ann[0:math.ceil(len(f_i)*0.2)])
  train_x=np.asarray(images[len(test_x):len(test_x)+math.ceil(len(f_i)*0.6)])
  #train_file=f_i[len(test_x):len(test_x)+math.ceil(len(f_i)*0.6)]
  train_y=np.array(ann[len(test_x):len(test_x)+math.ceil(len(f_i)*0.6)])
  val_x=np.asarray(images[len(test_x)+len(train_x):])
  #val_file=f_i[len(test_x)+len(train_x):]
  val_y=np.array(ann[len(test_x)+len(train_x):])
  train_y = to_categorical(train_y, 24)
  test_y = to_categorical(test_y, 24) 
  val_y= to_categorical(val_y, 24)
  print(f"Split a total of {len(f_i)} samples into three sets with {len(train_x)} training, {len(val_x)} validation and {len(test_x)} test samples.")
  return test_x,test_y,train_x,train_y,val_x,val_y,test_file


x_test,y_test,x_train,y_train,x_val,y_val,test_file=split_train_test_validation(path_ann,path_images)


Sorting both images and annotations...
Processing the images...
Processing the annotations
Split a total of 16940 samples into three sets with 10164 training, 3388 validation and 3388 test samples.


In [1]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import Input


# RNN model (RMSE: 0.164 after 10 epochs)
model = tf.keras.Sequential([
    layers.Conv2D(16, (5, 5), activation='relu', input_shape=(100, 100,1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (5, 5), activation='relu',kernel_regularizer=keras.regularizers.l2(0.005)), 
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(16, (5, 5), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (5, 5), activation='relu',kernel_regularizer=keras.regularizers.l2(0.005)),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu',kernel_regularizer=keras.regularizers.l2(0.005)),
    layers.Dense(24, activation='softmax')
])
model.summary()
input_shape=(288,432)
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.055),loss='categorical_crossentropy',metrics=['acc'])
history = model.fit(x_train, y_train, epochs=75, batch_size=132,validation_data=(x_val, y_val),callbacks=[es,mc])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 16)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 16)        6416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 22, 22, 16)       0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 22, 22, 16)       64        
 ormalization)                                                   
                                                        

NameError: ignored

In [ ]:
pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='test')

In [19]:
def get_accuracy(model, true_x, true_y): 
    res = model.predict(true_x)
    res = np.argmax(res, axis=-1)
    right = 0
    for i in range(len(true_y[:, 0])):
        r=res[i]
        t=np.where(true_y[i,:]==1)[0]
        if r==t:
          
          right+=1
        elif r==(t+7)% (24//2):
          right+=0.5
        elif r==(t-3) or r==(t+9)%24 or r==(t+3)%(24//2):
          right+=0.25
        elif (r==(t+12)%24):
          right+=0.1
        else:
            pass
    tot = len(true_y[:,0])
    print('True - total', right, tot)
    print('acc: {}'.format((right/tot)))

def get_dataset(label,img,test_file):
  f_l=[f for f in listdir("Annotations") if isfile(join("Annotations", f))]
  f_i=[f for f in listdir("Image_spec2") if isfile(join("Image_spec2", f))]
  f_l= np.asarray(f_l)
  f_i= np.asarray(f_i)
  for n in range(f_l.shape[0]):
    print(n)
    while True:
      s=splitext(f_l[n])[0]
      if "lerch" in s:
        s=s[:len(s)-6]
      res=np.where(f_i==f"{str(s)}.jpeg")
      if(n==res[0]):
        break
      f_i[n], f_i[res[0][0]] = f_i[res[0][0]], f_i[n]
  c=list(zip(f_l,f_i))
  giant_i=[]
  gtzan_i=[]
  gtzan_l=[]
  giant_l=[]
  lofi="LOFI"
  words=['blues','classical','country','disco','hiphop','jazz','metal','rock','reggae','pop']
  for f in f_i:
    if f in test_file: 
      if lofi in f:
        idx=np.where(f_i == f)
        giant_i.append(f)
        giant_l.append(f_l[idx])
      for w in words:
        if w in f:
          idx=np.where(f_i == f)
          gtzan_i.append(f)
          gtzan_l.append(f_l[idx])
  gt_i=[]
  gt_l=[]
  gi_i=[]
  gi_l=[]
  for g1 in gtzan_i:
    gt_i.append(np.asarray(ImageOps.grayscale(Image.open(f"Image_spec2/{g1}").resize((100,100)))))
  for g2 in giant_i:  
    gi_i.append(np.asarray(ImageOps.grayscale(Image.open(f"Image_spec2/{g2}").resize((100,100)))))
  for g1 in gtzan_l:
    with open(f"Annotations/{g1[0]}") as f:
      gt_l.append(int(f.readlines()[0]))
  for g2 in giant_l:
    with open(f"Annotations/{g2[0]}") as f:
      gi_l.append(int(f.readlines()[0]))
  gt_i=np.asarray(gt_i)
  gt_l=np.asarray(gt_l)
  gi_i=np.asarray(gi_i)
  gi_l=np.asarray(gi_l)
  return gt_i,gt_l,gi_i,gi_l
gtzan_i,gtzan_l,giant_i,giant_l=get_dataset(path_ann,path_images,test_file)
get_accuracy(model,giant_i,giant_l)

Output streaming troncato alle ultime 5000 righe.
changed 0_2nclassical.00032.jpeg with ['0_62666332.LOFI_3.jpeg']
14437
changed 0_2n3767408.LOFI_4.jpeg with ['0_24983645.LOFI_2.jpeg']
14438
changed 0_6n4364668.LOFI_2.jpeg with ['0_2n1007941.LOFI_3.jpeg']
14439
changed 0_63963180.LOFI_2.jpeg with ['0_63367555.LOFI_2.jpeg']
14440
changed 0_23772753.LOFI_1.jpeg with ['0_6n3930672.LOFI_2.jpeg']
14441
changed 2025670.LOFI_1.jpeg with ['0_6n1406309.LOFI_2.jpeg']
14442
changed 4452003.LOFI_1.jpeg with ['rock.00013.jpeg']
14443
changed 0_21939732.LOFI_2.jpeg with ['0_64508323.LOFI_4.jpeg']
14444
changed 0_24960012.LOFI_3.jpeg with ['0_6nreggae.00019.jpeg']
14445
changed 0_65103693.LOFI_3.jpeg with ['0_64284574.LOFI_1.jpeg']
14446
changed 0_63545728.LOFI_2.jpeg with ['0_23568459.LOFI_2.jpeg']
14447
changed 0_2npop.00077.jpeg with ['1803909.LOFI_4.jpeg']
14448
changed 0_2n4957129.LOFI_2.jpeg with ['0_6rock.00020.jpeg']
14449
changed reggae.00087.jpeg with ['classical.00005.jpeg']
14450
changed 

NameError: ignored